In [1]:
import geopandas as gpd
import pandas as pd
import folium
from folium import Choropleth
from folium import GeoJson, GeoJsonTooltip
from pygris import tracts
import branca
import os

In [8]:
import os

# HTML content with maps and insights
html_content = """
<!DOCTYPE html>
<html>
<head>
    <title>Multiple Metric Maps with Detailed Insights</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 20px;
            background-color: #f9f9f9;
        }
        .title {
            text-align: center;
            font-size: 24px;
            color: #333;
            margin-bottom: 20px;
        }
        .row {
            display: flex;
            justify-content: space-between;
            align-items: stretch;
            margin-bottom: 40px;
            height: 400px;
        }
        .map-container {
            width: 60%;
            border: 1px solid #ddd;
            border-radius: 8px;
            background-color: white;
            box-shadow: 0 2px 8px rgba(0, 0, 0, 0.1);
            padding: 0;
        }
        .map {
            width: 100%;
            height: 100%;
        }
        .insights-container {
            width: 40%;
            font-size: 14px;
            color: #333;
            line-height: 1.5;
            background-color: #fff;
            padding: 15px;
            border: 1px solid #ddd;
            border-radius: 8px;
            box-shadow: 0 2px 8px rgba(0, 0, 0, 0.1);
        }
    </style>
</head>
<body>
    <div class="title">Metric Maps with Detailed Insights</div>

    <!-- % Households with SNAP -->
    <div class="row">
        <div class="map-container">
            <iframe src="Pct_hh_snap_map.html" class="map" frameborder="0"></iframe>
        </div>
        <div class="insights-container">
            <h2>Insights: % Households with SNAP</h2>
            <p>This map highlights the geographic concentration of households relying on Supplemental Nutrition Assistance Program (SNAP) benefits, which indicate economic vulnerability and food insecurity.</p>
            <p>Areas with darker shades often correlate with higher unemployment rates, low median household incomes, and limited access to stable employment opportunities. Regions heavily reliant on SNAP may also face challenges such as higher poverty levels, limited access to nutritious food, and overburdened local food systems.</p>
            <p>Interventions such as workforce development programs and initiatives to improve access to affordable, nutritious food could help reduce dependence on government assistance and improve quality of life in these areas.</p>
        </div>
    </div>

    <!-- % Without Health Insurance -->
    <div class="row">
        <div class="map-container">
            <iframe src="Pct_noHealthInsurance_map.html" class="map" frameborder="0"></iframe>
        </div>
        <div class="insights-container">
            <h2>Insights: % Without Health Insurance</h2>
            <p>This map highlights regions with high proportions of uninsured individuals, often due to cost barriers or employment-related gaps. Darker areas correlate with poorer health outcomes and higher incidences of chronic illnesses.</p>
            <p>Lack of insurance often limits access to preventive care, resulting in higher healthcare costs in emergencies. Expanding Medicaid coverage, subsidizing insurance programs, and providing community health education could significantly improve access to healthcare and health outcomes.</p>
        </div>
    </div>

    <!-- % Without Internet Access -->
    <div class="row">
        <div class="map-container">
            <iframe src="Pct_internet_map.html" class="map" frameborder="0"></iframe>
        </div>
        <div class="insights-container">
            <h2>Insights: % Without Internet Access</h2>
            <p>This map illustrates the digital divide, with darker areas indicating limited access to broadband. These regions face challenges in education, remote work, and telemedicine, often exacerbating existing inequalities.</p>
            <p>Investments in broadband infrastructure, community Wi-Fi zones, and digital inclusion programs could address this gap and enhance access to essential online services.</p>
        </div>
    </div>

    <!-- % Without Device Access -->
    <div class="row">
        <div class="map-container">
            <iframe src="Pct_deviceAccess_map.html" class="map" frameborder="0"></iframe>
        </div>
        <div class="insights-container">
            <h2>Insights: % Without Device Access</h2>
            <p>Households without access to devices such as laptops or smartphones are highlighted in this map. Limited device access restricts education, job opportunities, and online services.</p>
            <p>Solutions include device distribution programs and affordable device rental initiatives for underserved communities.</p>
        </div>
    </div>

    <!-- % Minority Population -->
    <div class="row">
        <div class="map-container">
            <iframe src="Pct_minority_map.html" class="map" frameborder="0"></iframe>
        </div>
        <div class="insights-container">
            <h2>Insights: % Minority Population</h2>
            <p>This map reflects the distribution of minority populations, often concentrated in urban or rural pockets. These areas may face disparities in housing, education, and healthcare access.</p>
            <p>Targeted investments in minority-owned businesses and tailored community programs could promote equity and inclusion in these regions.</p>
        </div>
    </div>

    <!-- % High School Graduates -->
    <div class="row">
        <div class="map-container">
            <iframe src="Pct_HsGrad_map.html" class="map" frameborder="0"></iframe>
        </div>
        <div class="insights-container">
            <h2>Insights: % High School Graduates</h2>
            <p>The percentage of high school graduates in an area indicates educational attainment and correlates with job opportunities. Lower percentages in lighter areas highlight educational disparities.</p>
            <p>Expanding funding for schools, vocational training, and adult education programs could help bridge the gap and boost economic opportunities.</p>
        </div>
    </div>

    <!-- % Non-Native Population -->
    <div class="row">
        <div class="map-container">
            <iframe src="Pct_nonNative_map.html" class="map" frameborder="0"></iframe>
        </div>
        <div class="insights-container">
            <h2>Insights: % Non-Native Population</h2>
            <p>This map highlights areas with significant non-native populations, reflecting migration trends. These populations contribute to cultural diversity but may face integration challenges.</p>
            <p>Language training, job support, and community-building initiatives can support their inclusion and success.</p>
        </div>
    </div>

    <!-- Income Inequality (GINI Index) -->
    <div class="row">
        <div class="map-container">
            <iframe src="Income_Inequality_(GINI_Index)_map.html" class="map" frameborder="0"></iframe>
        </div>
        <div class="insights-container">
            <h2>Insights: Income Inequality (GINI Index)</h2>
            <p>The GINI Index highlights income inequality, with darker regions representing greater disparities. High inequality often correlates with reduced social mobility and higher health disparities.</p>
            <p>Policies promoting equitable access to housing, education, and progressive taxation could mitigate these challenges.</p>
        </div>
    </div>

    <!-- % Without Transportation Access -->
    <div class="row">
        <div class="map-container">
            <iframe src="Pct_transportAccess_map.html" class="map" frameborder="0"></iframe>
        </div>
        <div class="insights-container">
            <h2>Insights: % Without Transportation Access</h2>
            <p>This map highlights areas where residents lack reliable transportation, limiting access to jobs, education, and healthcare.</p>
            <p>Expanding public transit networks, ride-sharing programs, and affordable transportation options could reduce isolation and improve connectivity in these regions.</p>
        </div>
    </div>
    
    <!-- Unemployment Rate -->
    <div class="row">
        <div class="map-container">
            <iframe src="Unemployment_Rate_map.html" class="map" frameborder="0"></iframe>
        </div>
        <div class="insights-container">
            <h2>Insights: Unemployment Rate</h2>
            <p>The Unemployment Rate Map visualizes the distribution of unemployment across different geographic regions. Darker regions on the map signify higher unemployment rates, indicating areas with greater economic challenges.</p>
            <p>Unemployment levels are a critical indicator of economic health. Higher unemployment rates often correlate with reduced household incomes, limited access to services, and increased reliance on public assistance programs. Persistent unemployment in certain regions can create economic stagnation, reducing investments and infrastructure development.</p>
            <p>Potential solutions to address high unemployment include workforce training programs, local job creation initiatives, and targeted investments in industries suited to the region's strengths. Additionally, improving transportation access and affordable housing options can help connect unemployed individuals to employment opportunities.</p>
        </div>
    </div>
    
</body>
</html>
"""

# Define the file path
output_path = "C:/Users/RakshitMadan/OneDrive - Tech Impact/Career Mobility/Maps_3/multiple_metric_maps_with_insights.html"

# Ensure the folder exists
#os.makedirs("Maps_3", exist_ok=True)

# Save the HTML content to the file
with open(output_path, "w") as file:
    file.write(html_content)

print(f"Consolidated HTML file saved at: {output_path}")


Consolidated HTML file saved at: C:/Users/RakshitMadan/OneDrive - Tech Impact/Career Mobility/Maps_3/multiple_metric_maps_with_insights.html
